In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn import svm
%matplotlib inline

In [ ]:
data = pd.read_csv('../input/train.csv')
train_data = data.drop('is_duplicate', axis=1)
label_data = data.ix[:, 'is_duplicate']

In [ ]:
test_data = pd.read_csv('../input/test.csv')

In [ ]:
stops = set(stopwords.words("english"))

In [ ]:
stops

In [ ]:

full_stop = (',', '!', '.', ';')

In [ ]:
label_data.head(10)

In [ ]:
def sent_to_word(row):
    if not isinstance(row, str):
        row = str(row)
    sen_split = row.split(' ')
    split_list1 = [x.lower() for x in sen_split if not x.endswith(full_stop)]
    split_list2 = [''.join(list(map(str, x))[:-1]).lower() for x in sen_split if x.endswith(full_stop)]
    split_list = split_list1 + split_list2
    return split_list
def sent_len(row):
    return len(sent_to_word(row))
def overlap_words(row):
    inter_set = set(row.word_list1).intersection(set(row.word_list2))
    stops = set(stopwords.words("english"))
    filtered_set = [word for word in inter_set if word not in stops]
    return filtered_set

In [ ]:
def make_input(train_data):
    train_data['sent_diff'] = train_data.question1.astype(str).apply(sent_len) - train_data.question2.astype(str).apply(sent_len)
    train_data['word_list1'] = train_data.question1.apply(sent_to_word)
    train_data['word_list2'] = train_data.question2.astype(str).apply(sent_to_word)
    train_data['overlap'] = train_data.apply(overlap_words, axis=1)
    train_data['len_overlap'] = train_data['overlap'].apply(len)
    train_data['len_over_ratio'] = train_data['len_overlap']/(train_data['word_list1'].apply(len)+train_data['word_list2'].apply(len))
    train_data['words'] = train_data['word_list1'] + train_data['word_list2']
    return train_data

    

In [ ]:
input_train = make_input(train_data)

In [ ]:
input_test = make_input(test_data)

In [ ]:
df1, df2 = input_train.words, input_test.words
vocab = pd.concat([df1, df2], ignore_index=True)

In [ ]:
words_pool = {}
for i in range(vocab.shape[0]):
    for word in vocab[i]:
        if word not in words_pool.keys():
            words_pool[word] = 1
        else:
            words_pool[word] += 1

In [ ]:
words_pool

In [ ]:
from collections import Counter
voc = []
for row in vocab:
    voc += row
Counter(voc)

In [ ]:
input_train.head()

In [ ]:
def weight(row):
    w = 0
    for word in row:
        score = 1 + np.log(words_pool[word])
        w += 1/score
    return w
        
        
input_train['weight'] = input_train['overlap'].apply(weight)
input_train = input_train.ix[:,['sent_diff','len_over_ratio','weight']]


 
input_test['weight'] = input_test['overlap'].apply(weight)
input_test = input_test.ix[:,['sent_diff','len_over_ratio','weight']]


In [ ]:
from sklearn.metrics import log_loss

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import xgboost as xgb

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.025
params['max_depth'] = 4

d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

watchlist = [(d_train, 'train'), (d_test, 'valid')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)